# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [1]:
# Use the following data for this assignment:

import pandas as pd
import numpy as np

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650),
                   np.random.normal(44000,90000,3650)],
                  index=[1992,1993,1994,1995,1996])
df


,0,1,2,3,4,5,6,7,8,9,...,3640,3641,3642,3643,3644,3645,3646,3647,3648,3649
1992,-8941.531897,127788.667612,-71887.743011,-79146.060869,425156.114501,310681.166595,50581.575349,88349.230566,185804.513522,281286.947277,...,171938.760289,150650.759924,203663.976475,-377877.158072,-197214.093861,24185.008589,-56826.729535,-67319.766489,113377.299342,-4494.878538
1993,-51896.094813,198350.518755,-123518.252821,-129916.759685,216119.147314,49845.883728,149135.648505,62807.672113,23365.577348,-109686.264981,...,-44566.520071,101032.122475,117648.199945,160475.622607,-13759.888342,-37333.493572,103019.841174,179746.127403,13455.493990,34442.898855
1994,152336.932066,192947.128056,389950.263156,-93006.152024,100818.575896,5529.230706,-32989.370488,223942.967178,-66721.580898,47826.269111,...,165085.806360,74735.174090,107329.726875,199250.734156,-36792.202754,-71861.846997,26375.113219,-29328.078384,65858.761714,-91542.001049
1995,-69708.439062,-13289.977022,-30178.390991,55052.181256,152883.621657,12930.835194,63700.461932,64148.489835,-29316.268556,59645.677367,...,-13901.388118,50173.686673,53965.990717,4128.990173,72202.595138,39937.199964,139472.114293,59386.186379,73362.229590,28705.082908
1996,37870.332327,132060.164225,12030.042669,4518.971169,-26626.380691,15621.807266,152637.198867,81240.146523,100628.613237,61271.280549,...,42362.773745,-90095.223998,86628.378857,99704.461562,6889.995245,79520.438460,53577.579121,141949.992619,331953.815588,-241097.454433


In [2]:
%matplotlib notebook

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as lines
import math
import numpy as np
import random

In [3]:
class DataAnalysis:
    def __init__(self, df):
        self.df = df
    
    def __call__(self):
        self.mean, self.err, self.max, self.min = self.get_statistics()
        return [self.mean, self.err, self.max, self.min]
    
    def get_statistics(self):
        # Analyze data and prepare for plotting
        mean = list(df.mean(axis = 1))
        std = df.std(axis = 1)    

        # Calculate confidence interval using: https://www.mathsisfun.com/data/confidence-interval.html
        # Z value for 95% interval is 1.96
        # mean +/- 1.96 * (std)/sqrt(n)
        n = df.shape[1]

        #1.96*(std[i]/math.sqrt(len(df)))
        err = [1.96*(i/math.sqrt(n)) for i in std]

        # Generate .95 mean interval - will be used for comparision with value of interest
        max = list(map(lambda x, y: x + y, mean, err))
        min = list(map(lambda x, y: x - y, mean, err))
        
        return mean, err, max, min
    

In [4]:
#results = DataAnalysis(df)
#results()

In [25]:
class DraggableLineChart:
    def __init__(self, df, mean, err, max, min):        
        
        # Statistics
        self.df = df
        self.mean = mean 
        self.err = err
        self.max = max
        self.min = min
        self.voi = self.mean[0]
        
        self.bar_chart = self.BarChart(self.voi, self.mean, self.min, self.max)
        
        self.connect()

    def connect(self):
        'connect to all the events'
        self.cidpress = self.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)  
    
    def on_press(self, event):
        'on button press see if the mouse is over the line and store some data'
        if event.inaxes != self.line.axes: return
        contains, attr = self.line.contains(event)
        if not contains: return        
        self.press = self.line.get_ydata()[0]

    def on_motion(self, event):
        'on motion move the line if the mouse is over the line'
        if self.press is None: return
        if event.inaxes != self.line.axes: return
    
        ypress = self.press
        dy = event.ydata - ypress
        self.line.set_ydata([ypress+dy])
        
        self.voi = self.line.get_ydata()[0]
        colors = self.cmap(self.bar_colors(self.voi, self.min, self.max))

        counter = 0
        for bar in self.barchart:
            bar.set_facecolor(color = colors[counter])
            counter += 1

        self.ax.set_title(f'Value of interest: {round(self.voi)}')
        
    def on_release(self, event):
        'on release reset the press data'
        self.press = None
        self.canvas.draw_idle()
    
    class BarChart:
        def __init__(self, voi, mean, min, max):
            # Plot
            self.err = err
            self.max = max
            self.min = min
            self.mean = mean
            self.voi = mean[0]
            
            self.fig, self.ax = plt.subplots(figsize=(6, 6))
            self.canvas = self.ax.get_figure().canvas
            
            self.cmap, self.norm, self.bounds = self.colors_on_the_fly()
            self.colors = self.cmap(self.bar_colors(self.voi, self.min, self.max))
            self.fig.colorbar(mpl.cm.ScalarMappable(
                norm=self.norm, cmap=self.cmap), ax=self.ax, ticks=self.bounds)
            self.barchart = self.ax.bar(df.index, self.mean, yerr = self.err, 
                                        edgecolor='k', color=self.colors)
            
            self.line = self.ax.axhline(self.voi, color='grey', picker=True, pickradius=5)
            self.ax.add_line(self.line)
            
            self.fig.suptitle('Press and drag the line to define value of interest', 
                          fontsize=14)
    
        def colors_on_the_fly(self):
            n = 11  # Define number of labels
            cmap = plt.get_cmap('RdBu_r')  # Define the colormap

            # Extract all colors from the map and create new color map
            cmaplist = [cmap(i) for i in range(cmap.N)]
            cmap = cmap.from_list('Blue to Red', cmaplist, cmap.N)

            # Generate a colormap index based on discrete intervals
            bounds = np.linspace(0,1,n+1).round(decimals=2)
            norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

            return cmap, norm, bounds

        def bar_colors(self, voi, min, max):
            col = []
            for i in range(0,len(min)):
                normalized_color = ((voi - min[i]) / (max[i] - min[i]))
                col.append(normalized_color)           
            return col
          

In [26]:
results = DataAnalysis(df)
results()
mean, err, max, min = results()
interactive_chart = DraggableLineChart(df, mean, err, max, min)
plt.show()


<IPython.core.display.Javascript object>

AttributeError: 'BarChart' object has no attribute 'err'